# Regressão Linear Simples - Trabalho

## Estudo de caso: Seguro de automóvel sueco

Agora, sabemos como implementar um modelo de regressão linear simples. Vamos aplicá-lo ao conjunto de dados do seguro de automóveis sueco. Esta seção assume que você baixou o conjunto de dados para o arquivo insurance.csv, o qual está disponível no notebook respectivo.

O conjunto de dados envolve a previsão do pagamento total de todas as reclamações em milhares de Kronor sueco, dado o número total de reclamações. É um dataset composto por 63 observações com 1 variável de entrada e 1 variável de saída. Os nomes das variáveis são os seguintes:

1. Número de reivindicações.
2. Pagamento total para todas as reclamações em milhares de Kronor sueco.

Voce deve adicionar algumas funções acessórias à regressão linear simples. Especificamente, uma função para carregar o arquivo CSV chamado *load_csv ()*, uma função para converter um conjunto de dados carregado para números chamado *str_column_to_float ()*, uma função para avaliar um algoritmo usando um conjunto de treino e teste chamado *split_train_split ()*, a função para calcular RMSE chamado *rmse_metric ()* e uma função para avaliar um algoritmo chamado *evaluate_algorithm()*.

Utilize um conjunto de dados de treinamento de 60% dos dados para preparar o modelo. As previsões devem ser feitas nos restantes 40%. 

Compare a performabce do seu algoritmo com o algoritmo baseline, o qual utiliza a média dos pagamentos realizados para realizar a predição ( a média é 72,251 mil Kronor).


# Bibliotecas usadas

In [14]:
import pandas as pd
import numpy as np
import random
from sklearn.isotonic import IsotonicRegression

In [2]:
def load_csv(caminho, separador):
    return pd.read_csv(caminho, sep = separador, names = ['x','y'])

In [3]:
def str_column_to_float (df, column):
    try:
        df[column] = np.float64(df[column])
    except:
        df[column] = np.float64( [x.replace(",",".") for x in list(df[column])] )

In [4]:
def covariance(xl, mean_x, yl, mean_y):
    covar = 0.0
    for x,y in zip(xl, yl):
        covar += (x - mean_x) * (y - mean_y)
    return covar

In [5]:
def coefficients(dataset):
    x = np.array(dataset['x'])
    y = np.array(dataset['y'])
    x_mean, y_mean = np.mean(x), np.mean(y)
    b1 = covariance(x, x_mean, y, y_mean) / np.var(x)
    b0 = y_mean - b1 * x_mean
    return b0, b1

In [6]:
def rmse_metric (predicao, alvo):
    soma = []
    for pred, alv in zip(predicao, alvo):
        soma.append( (pred-alv)**2 )
    return np.sqrt(np.mean(soma))

In [7]:
def regressãoLinearBaseLine(train, test):
    predictions = []
    b0, b1 = coefficients(train)
    for x in np.array(test['x']):
        ypred = b0 + b1 * x
        predictions.append(ypred)
    return np.array(predictions)

In [16]:
def Isotonic_Regression(train, test):
    xt = np.array(train['x'])
    yt = np.array(train['y'])
    
    ir = IsotonicRegression()
    
    t = ir.fit(xt, yt)
    
    return t.predict(np.array(test['x']))

In [17]:
def split_train_split (train, test, algoritmo):
    
    predicoes = algoritmo(train, test)
    
    alvo = np.array(test['y'])
    
    return rmse_metric(predicoes, alvo)
    

In [22]:
def splitDataset(df, splitRatio):
    msk = np.random.rand(len(df)) < splitRatio
    train = df[msk]
    test = df[~msk]
    return train, test

In [25]:
def evaluate_algorithm():
    insurance = load_csv('insurance.csv', ',')
    
    str_column_to_float(insurance,'x')
    str_column_to_float(insurance,'y')
    
    train_insurance, test_insurance = splitDataset(insurance, 0.6)
    
    resultadoInsuranceLR = split_train_split(train_insurance, test_insurance, regressãoLinearBaseLine)
    resultadoInsuranceIR = split_train_split(train_insurance, test_insurance, Isotonic_Regression)
    
    print("Resultados: Base Line {} Isotonic Regression {}".format(resultadoInsuranceLR, resultadoInsuranceIR))
    
    return 0

In [26]:
evaluate_algorithm()

Resultados: Base Line 1851.7424892310585 Isotonic Regression 43.9234541360834


0